**Sometimes you need to kill the session in order to read new files on Google Drive**

In [0]:
# !google-drive-ocamlfuse drive
!kill -9 -1

**Mount Google Drive**

In [0]:
from google.colab import drive
import os
drive.mount('/content/drive')

**Folder you need to create on Google Drive: datasets\videos**

In [0]:
data_path = os.path.sep + os.path.join('content', 'drive', 'My\ Drive', 'datasets', 'videos').replace('\\', '')

**Install some packages**

In [0]:
!pip install turicreate
!pip install face_recognition
!pip install mhyt
!pip install tqdm

**Define Kaggle API settings**

In [0]:
# see more details in the course first lecture
!mkdir /root/.kaggle/
import json
import os

# Installing the Kaggle package
!pip install kaggle 

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"",
"key":""}
  # TODO: REMOVE THIS LINE!!!

# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

## Homework Assignment 9

**Question 1:** Select a short video with at least 3 persons and create a new movie from this video with a face tracker (each person’s face needs to be tracked by a rectangle of a different color) (50pt). See, for example, the video in: https://github.com/ageitgey/face_recognition

**Bonus:** Select a video with at least two animals (dog/cat/lion/tiger...), and create a video with an animal tracker (10pt).

**Upload images for each persons we want to recognize (Google Drive --> datasets\videos\images**)

In [0]:
import cv2
import face_recognition  # requires GPUs in Colab

known_faces = []
images_folder = os.path.join(data_path, "images")
names_images = {
  'conan obrien': '3000.jpeg',
  'James Parsons': '46844.jpg',
  'Simon Helberg': 'Helbergrl.png'
}
names_colors = {
  'conan obrien': (255, 0, 0),
  'James Parsons': (255, 255, 0),
  'Simon Helberg': (255, 0, 255)
}
names = []

for name, pic in names_images.items():
  names.append(name)
  loaded_pic = face_recognition.load_image_file(os.path.join(images_folder, pic))
  encoding_pic = face_recognition.face_encodings(loaded_pic)[0]
  known_faces.append(encoding_pic)

**Define the output files and object**

In [0]:
output_people_video = "output_people.avi"
# Create an output movie file (make sure resolution/frame rate matches input video!)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_movie = cv2.VideoWriter(output_people_video, fourcc, 5.0, (1920, 1080))

output_people_video_mp4 = "output_people.mp4"
output_people_video_gif = "output_people.gif"
![ -f $output_people_video_mp4 ] && rm $output_people_video_mp4
![ -f $output_people_video_gif ] && rm $output_people_video_gif

**Read the input video**

In [0]:
# Open the input movie file
video_name = "video_people.mp4"
input_people_video = os.path.join(data_path, video_name)
input_movie = cv2.VideoCapture(input_people_video)
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
print(f'Number of frames: {length}')

from IPython.display import HTML
from base64 import b64encode
mp4 = open(input_people_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width="1280" height="720" controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

**Read video frame by frame and draw rectangle and text for each person we recognized and write all these new frames**

In [0]:
from tqdm import tqdm

def get_faces_name(face_encodings, names):
  face_names = []
  for face_encoding in face_encodings:
      # See if the face is a match for the known face(s)
      match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.80)

      name = None
      for i in range(len(match)):
        if match[i]:
          name = names[i]
          break
      face_names.append(name)
  return face_names

def clean_session(input_movie, output_movie):
  input_movie.release()
  output_movie.release()
  cv2.destroyAllWindows()

def plot_box(top, right, bottom, left, name, names_colors, frame, cv2):
  if not name:
    return
  else:
    r = names_colors[name][0]
    g = names_colors[name][1]
    b = names_colors[name][2]

  # Draw a box around the face
  cv2.rectangle(frame, (left, top), (right, bottom), (r, g, b), 2)

  # Draw a label with a name below the face
  cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (r, g, b), cv2.FILLED)
  font = cv2.FONT_HERSHEY_SIMPLEX
  org = (left + 6, bottom - 6)
  thickness = 2
  color = (255, 255, 255)
  cv2.putText(frame, name, org, font, 0.5, color, thickness)

locations = []
encodings = []
face_names = []

number_of_frames = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
for frame_number in tqdm(range(number_of_frames)):
    ret, frame = input_movie.read()  # get the next frame
    if not ret:  # in case there is no frames anymore
        print("ret is None")
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    locations = face_recognition.face_locations(rgb_frame)
    encodings = face_recognition.face_encodings(rgb_frame, locations)
    face_names = get_faces_name(encodings, names)

    for (top, right, bottom, left), name in zip(locations, face_names):
      plot_box(top, right, bottom, left, name, names_colors, frame, cv2)

    output_movie.write(frame)  # Write the resulting image to the output video file

clean_session(input_movie, output_movie)
print('Done!')

**Convert AVI format to MP4**

In [0]:
!ffmpeg -i $output_people_video $output_people_video_mp4

**Present the output video (I ignored the woman on the left side since she is not a celebrity)**

In [0]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(output_people_video_mp4,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width="1280" height="720" controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

**Convert AVI to GIF**

In [0]:
!ffmpeg -i $output_people_video $output_people_video_gif

**Present the GIF output (from my Github):**

In [0]:
from IPython.display import Image
Image(url='https://github.com/nevoit/Analyzing-Big-Data-Course/raw/master/figures/hw_9_gif.gif')

**Question 2:** Select a collection of connected images. Create a graph of the links among images or objects in images (42pt). Use graph algorithms to discover interesting insights regarding the images. See, for example, [Figure 2](https://arxiv.org/pdf/1509.00568.pdf) (8pt).

**Connect to kaggle and download the [simpsons datasets](https://www.kaggle.com/alexattia/the-simpsons-characters-dataset)**

In [0]:
DATASET_NAME = 'simpsons'
KAGGLE_REPO = 'alexattia/the-simpsons-characters-dataset'

In [0]:
# searching for the dataset
!rm -r ./datasets/$DATASET_NAME/
!mkdir -p ./datasets/$DATASET_NAME/data
!mkdir -p ./datasets/$DATASET_NAME/zip
!mkdir -p ./datasets/$DATASET_NAME/sql
!ls datasets/$DATASET_NAME/data

!kaggle datasets list -s $DATASET_NAME
# download the dataset from Kaggle and unzip it
!kaggle datasets download $KAGGLE_REPO -p ./datasets/$DATASET_NAME/zip
!unzip ./datasets/$DATASET_NAME/zip/*.zip -d ./datasets/$DATASET_NAME/data
!ls ./datasets/$DATASET_NAME/zip/
!ls ./datasets/$DATASET_NAME/data/
!ls ./datasets/$DATASET_NAME/sql/

**Read the dataset using turicreate**

In [0]:
import turicreate as tc
import turicreate.aggregate as agg
DATASET_PATH = f"./datasets/{DATASET_NAME}/data/simpsons_dataset"

data_perc = 0.05
seed = 2020
sf = tc.image_analysis.load_images(DATASET_PATH, with_path=True).sample(data_perc, seed=seed)
sf

**Extract the character name using the path column**

In [0]:
sf['character'] = sf['path'].apply(lambda p: p.split('/')[-2])
sf

In [0]:
sf[:5].explore()

**Let's calculate the feature vector of each image:**

In [0]:
from turicreate.toolkits import _image_feature_extractor, _pre_trained_models

def get_images_features_vector(dataset, target, feature="image", model_name='resnet-50'):
    #ptModel = _pre_trained_models.MODELS[model_name]()
    ptModel = tc.toolkits._pre_trained_models.ResNetImageClassifier()
    feature_extractor = tc.toolkits._image_feature_extractor._create_feature_extractor(model_name)
    extracted_features = tc.SFrame({
        target: dataset[target],
        '__image_features__': feature_extractor.extract_features(dataset, feature),
        })
    dataset['image_features'] = extracted_features['__image_features__']
    return dataset

sf = get_images_features_vector(sf, 'character')
sf

In [0]:
sf.save('./simpsons.sframe')

In [0]:
model = tc.image_similarity.create(sf)

**The TuriCreate API supports this type of [similar image matching](https://apple.github.io/turicreate/docs/userguide/image_similarity/). Let's use it directly:**

In [0]:
similarity_graph = model.similarity_graph(k=2)
similarity_graph.summary()

**Let's use Networkx and Cytoscape to draw one of the communities of the graph:**

In [0]:
import matplotlib.image as mpimg
import networkx as nx

g = nx.Graph()
for v in similarity_graph.vertices['__id']:
    g.add_node(v, attr_dict={'path':sf[v]['path'], 'character': sf[v]['character']})
for e in similarity_graph.edges:
    g.add_edge(e["__src_id"], e["__dst_id"])
nx.info(g)

**Find communities in graph**

In [0]:
from networkx.algorithms.community import greedy_modularity_communities
c = list(greedy_modularity_communities(g))
len(c)

In [0]:
d = {i: len(c[i]) for i in range(len(c))}
d

**Create a subgraph using the first community**

In [0]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
from numpy import sqrt
import glob

communities_num = 0
h = g.subgraph(c[communities_num])
nx.info(h)

**Plot the selected community (the subgraph) with images instead nodes.**

In [0]:
fig=plt.figure(figsize=(30,30))
ax=plt.subplot(111)
ax.set_aspect('equal')
pos=nx.kamada_kawai_layout(h, scale=3)

# draw with images on nodes
nx.draw_networkx(h,pos,width=3,edge_color="r",alpha=0.6)
ax=plt.gca()
fig=plt.gcf()
trans = ax.transData.transform
trans2 = fig.transFigure.inverted().transform
imsize = 0.03 # this is the image size
node_dict = {}
for n, attr in h.nodes(data=True):
    char_name = attr['attr_dict']['character']
    path_to_image = attr['attr_dict']['path']
    # print(f'Character {char_name}, Node: {n}')
    (x,y) = pos[n]
    xx,yy = trans((x,y)) # figure coordinates
    xa,ya = trans2((xx,yy)) # axes coordinates
    a = plt.axes([xa-imsize/2.0,ya-imsize/2.0, imsize, imsize ])
    a.imshow(mpimg.imread(path_to_image))
    a.set_aspect('equal')
    a.axis('off')
    node_dict[n] = (char_name, path_to_image)

**Use graph algorithms to discover interesting insights regarding the images**

In [0]:
import operator
d = nx.pagerank(h)
pg = max(dict(d).items(), key=operator.itemgetter(1))   
pg

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

max_node_pg = pg[0]
hero_name_pg = node_dict[max_node_pg][0].replace('_', ' ')
print(f'The hero with the highest pagerank ({pg[1]}) is "{hero_name_pg}".')
plt.imshow(mpimg.imread(node_dict[max_node_pg][1]))


**According to Closeness Centrality who is the most central hero:**

In [0]:
d = nx.closeness_centrality(h) # can take some time to run

cen = max(dict(d).items(), key=operator.itemgetter(1))
cen

In [0]:
max_node_cen = cen[0]
hero_name_cen = node_dict[max_node_cen][0].replace('_', ' ')
print(f'Closeness Centrality ({cen[1]}), who is the most central hero, is "{hero_name_cen}".')

plt.imshow(mpimg.imread(node_dict[max_node_cen][1]))